L1_demo_02_function_calling
This demo explores how to use function calling with OpenAI’s chat models while maintaining a memory layer. The system identifies when to use a tool (function), extracts arguments, executes the tool, and feeds the result back into the conversation, enabling a full loop of reasoning and tool use.



Initial setup

In [1]:
import json
from typing import List, Dict, Literal
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage

from openai import OpenAI
client = OpenAI()  # reads OPENAI_API_KEY from env

# Auth + connectivity check
models = client.models.list()
print("Model count:", len(models.data))

Model count: 84


1. Memory Class Recap and Enhancement
A custom Memory class is used to:
Add new messages.
Fetch all messages.
Fetch the last message.
Reset the memory.

In [2]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []
    
    def add_message(self, 
                    role: Literal['user', 'system', 'assistant', 'tool'], 
                    content: str,
                    tool_calls: dict=dict(),
                    tool_call_id=None)-> None:

        message = {
            "role": role,
            "content": content,
            "tool_calls": tool_calls,
        }

        if role == "tool":
            message = {
                "role": role,
                "content": content,
                "tool_call_id": tool_call_id,
            }

        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    # A new method
    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    # A new method
    def reset(self) -> None:
        self._messages = []

In [3]:
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

In [4]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}}]

In [5]:
memory.reset()

In [6]:
memory.get_messages()

[]

3. Enhanced Chat Function
chat_with_tools is created to:
Accept user input and available tools.
Pass the tools and memory to the model.
Handle tool call outputs.

In [7]:
def chat_with_tools(user_question:str=None, 
                    memory:Memory=None, 
                    model:str="gpt-4o-mini", 
                    temperature=0.0, 
                    tools=None)-> str:
    messages = [{"role": "user", "content": user_question}]
    if memory:
        if user_question:
            memory.add_message(role="user", content=user_question)
        messages = memory.get_messages()        
    
    response = client.chat.completions.create(
        model = model,
        temperature = temperature,
        messages = messages,
        tools=tools, # Providing available tools to the model
    )
    
    ai_message = str(response.choices[0].message.content)
    tool_calls = response.choices[0].message.tool_calls # If the model decides to call a function
    
    if memory:
        memory.add_message(role="assistant", content=ai_message, tool_calls=tool_calls)
    
    return ai_message

In [8]:
chat_with_tools(
    "2 to the power of -5?",
    model="gpt-3.5-turbo",
)

'1/32 or 0.03125'

2. Function Calling Preparation
A custom function power(base, exponent) is defined for exponentiation.
This function is wrapped in a JSON schema tool description so the model can learn about its existence and parameters.

In [9]:
def power(base:float, exponent:float):
    """Exponentatiation: base to the power of exponent"""
    
    return base ** exponent

In [10]:
power(2, 3)

8

In [11]:
power(2, -5)

0.03125

In [12]:
tools = [{
    "type": "function",
    "function": {
        "name": "power",
        "description": "Exponentatiation: base to the power of exponent",
        "parameters": {
            "type": "object",
            "properties": {
                "base": {"type": "number"},
                "exponent": {"type": "number"}
            },
            "required": ["base", "exponent"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [13]:
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

In [14]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}}]

In [15]:
ai_message = chat_with_tools(
    "2 to the power of -5?",
    model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

In [16]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageFunctionToolCall(id='call_NdedJP9PsvPdj6JeE4scWkFu', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]}]

In [17]:
memory.last_message()['tool_calls']

[ChatCompletionMessageFunctionToolCall(id='call_NdedJP9PsvPdj6JeE4scWkFu', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]

In [18]:
tool_call_id = memory.last_message()['tool_calls'][0].id
tool_call_id

'call_NdedJP9PsvPdj6JeE4scWkFu'

4. Flow of Execution
The model receives a query like "2^-5".
Instead of responding directly, it outputs a tool call:
Function name: power
Arguments: base=2, exponent=-5
Developer extracts these arguments, executes the power function manually, and stores the result.
A new tool role message is added to the memory, linking it to the original tool call ID.

In [19]:
args = json.loads(memory.last_message()['tool_calls'][0].function.arguments)
args

{'base': 2, 'exponent': -5}

In [20]:
result = power(args["base"], args["exponent"])
result

0.03125

In [21]:
memory.add_message(role="tool", content=str(result), tool_call_id=tool_call_id)

In [22]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageFunctionToolCall(id='call_NdedJP9PsvPdj6JeE4scWkFu', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '0.03125',
  'tool_call_id': 'call_NdedJP9PsvPdj6JeE4scWkFu'}]

In [23]:
ai_message = chat_with_tools(
    model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

In [24]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageFunctionToolCall(id='call_NdedJP9PsvPdj6JeE4scWkFu', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '0.03125',
  'tool_call_id': 'call_NdedJP9PsvPdj6JeE4scWkFu'},
 {'role': 'assistant',
  'content': '2 to the power of -5 is 0.03125',
  'tool_calls': None}]

5. Final AI Response
After feeding back the tool’s result, the model is invoked again.
Now, the assistant produces a natural language response incorporating the tool’s output (e.g., "2 to the power of -5 is approximately 0.031").

6. Key Concepts Highlighted
The difference between standard responses and tool call behavior.
How function calling allows LLMs to extend their capabilities.
Importance of linking tool responses back via tool_call_id.
How memory tracks both user interactions and intermediate tool operations.

7. Conclusion
Function calling enables a much more structured and powerful interaction loop.
The memory structure supports complex multi-turn conversations that involve external tool use.
Proper handling of tool call and tool response messages is crucial for building advanced AI agents.